In [3]:
import sys
sys.path.append('/Users/alice/Code/pandas_helper')
sys.path.append('/Users/alice/Code/metaheuristics')

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math
import pickle

import univariate_analysis as ua
import bivariate_analysis as ba
import feature_selection as fs
from ipynb.fs.full.genetic_local_search import GeneticLocalSearch

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from scipy import stats

import warnings; warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'nltk'

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
# adding missing revenue, budgets, and runtimes
# from https://www.kaggle.com/enric1296/complete-guide-eda-feat-model
train.loc[train['id'] == 16,'revenue'] = 192864          # Skinning
train.loc[train['id'] == 90,'budget'] = 30000000         # Sommersby          
train.loc[train['id'] == 118,'budget'] = 60000000        # Wild Hogs
train.loc[train['id'] == 149,'budget'] = 18000000        # Beethoven
train.loc[train['id'] == 313,'revenue'] = 12000000       # The Cookout 
train.loc[train['id'] == 451,'revenue'] = 12000000       # Chasing Liberty
train.loc[train['id'] == 464,'budget'] = 20000000        # Parenthood
train.loc[train['id'] == 470,'budget'] = 13000000        # The Karate Kid, Part II
train.loc[train['id'] == 513,'budget'] = 930000          # From Prada to Nada
train.loc[train['id'] == 797,'budget'] = 8000000         # Welcome to Dongmakgol
train.loc[train['id'] == 819,'budget'] = 90000000        # Alvin and the Chipmunks: The Road Chip
train.loc[train['id'] == 850,'budget'] = 90000000        # Modern Times
train.loc[train['id'] == 1112,'budget'] = 7500000        # An Officer and a Gentleman
train.loc[train['id'] == 1131,'budget'] = 4300000        # Smokey and the Bandit   
train.loc[train['id'] == 1359,'budget'] = 10000000       # Stir Crazy 
train.loc[train['id'] == 1542,'budget'] = 10000000       # All at Once
train.loc[train['id'] == 1542,'budget'] = 15800000       # Crocodile Dundee II
train.loc[train['id'] == 1571,'budget'] = 4000000        # Lady and the Tramp
train.loc[train['id'] == 1714,'budget'] = 46000000       # The Recruit
train.loc[train['id'] == 1721,'budget'] = 17500000       # Cocoon
train.loc[train['id'] == 1865,'revenue'] = 25000000      # Scooby-Doo 2: Monsters Unleashed
train.loc[train['id'] == 2268,'budget'] = 17500000       # Madea Goes to Jail budget
train.loc[train['id'] == 2491,'revenue'] = 6800000       # Never Talk to Strangers
train.loc[train['id'] == 2602,'budget'] = 31000000       # Mr. Holland's Opus
train.loc[train['id'] == 2612,'budget'] = 15000000       # Field of Dreams
train.loc[train['id'] == 2696,'budget'] = 10000000       # Nurse 3-D
train.loc[train['id'] == 2801,'budget'] = 10000000       # Fracture
train.runtime[train.id == 391] = 86 #Il peor natagle de la meva vida
train.runtime[train.id == 592] = 90 #А поутру они проснулись
train.runtime[train.id == 925] = 95 #¿Quién mató a Bambi?
train.runtime[train.id == 978] = 93 #La peggior settimana della mia vita
train.runtime[train.id == 1256] = 92 #Cipolla Colt
train.runtime[train.id == 1542] = 93 #Все и сразу
train.runtime[train.id == 1875] = 86 #Vermist
train.runtime[train.id == 2151] = 108 #Mechenosets
train.runtime[train.id == 2499] = 108 #Na Igre 2. Novyy Uroven
train.runtime[train.id == 2646] = 98 #同桌的妳
train.runtime[train.id == 2786] = 111 #Revelation
train.runtime[train.id == 2866] = 96 #Tutto tutto niente niente


test.runtime[test.id == 4074] = 103 #Shikshanachya Aaicha Gho
test.runtime[test.id == 4222] = 93 #Street Knight
test.runtime[test.id == 4431] = 100 #Плюс один
test.runtime[test.id == 5520] = 86 #Glukhar v kino
test.runtime[test.id == 5845] = 83 #Frau Müller muss weg!
test.runtime[test.id == 5849] = 140 #Shabd
test.runtime[test.id == 6210] = 104 #Le dernier souffle
test.runtime[test.id == 6804] = 145 #Chaahat Ek Nasha..
test.runtime[test.id == 7321] = 87 #El truco del manco
test.loc[test['id'] == 3889,'budget'] = 15000000       # Colossal
test.loc[test['id'] == 6733,'budget'] = 5000000        # The Big Sick
test.loc[test['id'] == 3197,'budget'] = 8000000        # High-Rise
test.loc[test['id'] == 6683,'budget'] = 50000000       # The Pink Panther 2
test.loc[test['id'] == 5704,'budget'] = 4300000        # French Connection II
test.loc[test['id'] == 6109,'budget'] = 281756         # Dogtooth
test.loc[test['id'] == 7242,'budget'] = 10000000       # Addams Family Values
test.loc[test['id'] == 7021,'budget'] = 17540562       #  Two Is a Family
test.loc[test['id'] == 5591,'budget'] = 4000000        # The Orphanage
test.loc[test['id'] == 4282,'budget'] = 20000000       # Big Top Pee-wee

In [6]:
# more revenue corrections
# from https://www.the-numbers.com/
train.revenue.iloc[116] = 55287687
train.revenue.iloc[150] = 18000000
train.revenue.iloc[269] = 20018
train.revenue.iloc[280] = 10155691
train.revenue.iloc[347] = 72844
train.revenue.iloc[639] = 10557291
train.revenue.iloc[665] = 11514
train.revenue.iloc[1138] = 15545115
train.revenue.iloc[1161] = 272375
train.revenue.iloc[1198] = 104268727
train.revenue.iloc[1240] = 14683763
train.revenue.iloc[1254] = 651
train.revenue.iloc[1281] = 48977233
train.revenue.iloc[1354] = 3900927
train.revenue.iloc[1376] = 5245263
train.revenue.iloc[1479] = 127257
train.revenue.iloc[1754] = 1125910
train.revenue.iloc[1800] = 131188
train.revenue.iloc[1884] = 23693646
train.revenue.iloc[1948] = 204612
train.revenue.iloc[2117] = 3094699
train.revenue.iloc[2251] = 51119758
train.revenue.iloc[2255] = 6552255
train.revenue.iloc[2433] = 32330354

# from https://www.boxofficemojo.com/
train.revenue.iloc[152] = 241278
train.revenue.iloc[498] = 25317
train.revenue.iloc[1141] = 13746
train.revenue.iloc[1190] = 7660857
train.revenue.iloc[1346] = 5000000
train.revenue.iloc[2090] = 18369
train.revenue.iloc[2474] = 23462

In [7]:
# more runtime corrections
# from https://www.omdbapi.com/

train.runtime.iloc[390] = 86
train.runtime.iloc[591] = 86
train.runtime.iloc[1541] = 93
train.runtime.iloc[1874] = 92
train.runtime.iloc[2785] = 111
train.runtime.iloc[2865] = 96

test.runtime.iloc[1073] = 103
test.runtime.iloc[1221] = 91
test.runtime.iloc[1430] = 96
test.runtime.iloc[2844] = 83
test.runtime.iloc[2848] = 140
test.runtime.iloc[3803] = 140

In [8]:
# log scaling
train['budget_log'] = train.budget.apply(np.log1p)
test['budget_log'] = test.budget.apply(np.log1p)

train['revenue_log'] = train.revenue.apply(np.log1p)

train['popularity_log'] = train.popularity.apply(np.log1p)
test['popularity_log'] = test.popularity.apply(np.log1p)

In [9]:
# drop useless columns
train = train.drop(['imdb_id', 'poster_path', 'status'], axis = 1)
test = test.drop(['imdb_id', 'poster_path', 'status'], axis = 1)

In [10]:
# fill nans
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [11]:
# feature generation - categorical

train['has_homepage'] = train.homepage.apply(lambda x: x != '')
test['has_homepage'] = test.homepage.apply(lambda x: x != '')

train['has_tagline'] = train.tagline.apply(lambda x: x != '')
test['has_tagline'] = test.tagline.apply(lambda x: x != '')

train.release_date = pd.to_datetime(train.release_date)
train_fix_years = train.release_date[train.release_date.dt.year > 2018]
train_fix_years = train_fix_years.apply(lambda x: x.replace(x.year-100))
train.release_date.update(train_fix_years)
train['release_year'] = train.release_date.apply(lambda x: x.year)
train['release_month'] = train.release_date.apply(lambda x: x.month)
train['release_weekday'] = train.release_date.apply(lambda x: x.weekday())
train['release_decade'] = train.release_year.apply(lambda x: (x//10)*10)
test.release_date = pd.to_datetime(test.release_date)
test_fix_years = test.release_date[test.release_date.dt.year > 2018]
test_fix_years = test_fix_years.apply(lambda x: x.replace(x.year-100))
test.release_date.update(test_fix_years)
test['release_year'] = test.release_date.apply(lambda x: x.year)
test['release_month'] = test.release_date.apply(lambda x: x.month)
test['release_weekday'] = test.release_date.apply(lambda x: x.weekday())
test['release_decade'] = test.release_year.apply(lambda x: (x//10)*10)
for m in range(1,13):
    train['release_month_'+str(m)] = train.release_month.apply(lambda x: x==m)
    test['release_month_'+str(m)] = test.release_month.apply(lambda x: x==m)
for d in range(7):
    train['release_weekday_'+str(d)] = train.release_weekday.apply(lambda x: x==d)
    test['release_weekday_'+str(d)] = test.release_weekday.apply(lambda x: x==d)

# fill budget nans with median grouped by decade
train.budget = train.groupby(train.release_decade).budget.apply(lambda x: x.replace(0, x.median()))
test.budget = test.groupby(test.release_decade).budget.apply(lambda x: x.replace(0, x.median()))
train['budget_log'] = train.budget.apply(np.log1p)
test['budget_log'] = test.budget.apply(np.log1p)

train['has_collection'] = train.belongs_to_collection.apply(lambda x: x!='')
test['has_collection'] = test.belongs_to_collection.apply(lambda x: x!='')
train.belongs_to_collection = ua.get_value_col(train.belongs_to_collection, key='id')
test.belongs_to_collection = ua.get_value_col(test.belongs_to_collection, key='id')

train['cast_size'] = train.cast.apply(ua.how_many)
train['cast_freq'] = ua.get_freq_col(train.cast)
train['fame_score'] = train.cast_freq/train.cast_size
train.fame_score = train.fame_score.fillna(0)
test['cast_size'] = test.cast.apply(ua.how_many)
test['cast_freq'] = ua.get_freq_col(test.cast)
test['fame_score'] = test.cast_freq/test.cast_size
test.fame_score = test.fame_score.fillna(0)
char_df = ua.get_freq_df(column = train.cast, key='character')
for g in [0,1,2]:
    train['cast_gender_'+str(g)] = train.cast.apply(lambda x: ua.get_value_count(x, key='gender', value=g))
    train['cast_gender_'+str(g)] = train['cast_gender_'+str(g)]/train.cast_size
    test['cast_gender_'+str(g)] = test.cast.apply(lambda x: ua.get_value_count(x, key='gender', value=g))
    test['cast_gender_'+str(g)] = test['cast_gender_'+str(g)]/test.cast_size
for c in char_df.value[:15]:
    train['cast_character_'+c] = train.cast.apply(lambda x: c in x)
    test['cast_character_'+c] = test.cast.apply(lambda x: c in x)
    
train['crew_size'] = train.crew.apply(ua.how_many)
train['crew_director'] = ua.get_value_col(train.crew, k='job', v='Director')
train['crew_producer'] = ua.get_value_col(train.crew, k='job', v='Producer')
train['crew_executive_producer'] = ua.get_value_col(train.crew, k='job', v='Executive Producer')
test['crew_size'] = test.crew.apply(ua.how_many)
test['crew_director'] = ua.get_value_col(test.crew, k='job', v='Director')
test['crew_producer'] = ua.get_value_col(test.crew, k='job', v='Producer')
test['crew_executive_producer'] = ua.get_value_col(test.crew, k='job', v='Executive Producer')
for g in [0,1,2]:
    train['crew_gender_'+str(g)] = train.crew.apply(lambda x: ua.get_value_count(x, key='gender', value=g))
    train['crew_gender_'+str(g)] = train['crew_gender_'+str(g)]/train.crew_size
    test['crew_gender_'+str(g)] = test.crew.apply(lambda x: ua.get_value_count(x, key='gender', value=g))
    test['crew_gender_'+str(g)] = test['crew_gender_'+str(g)]/test.crew_size
crew_df = ua.get_freq_df(column=train.crew, key='name')
for c in crew_df.value[:15]:
    train['crew_name_'+c] = train.crew.apply(lambda x: c in x)
    test['crew_name_'+c] = test.crew.apply(lambda x: c in x)
job_df = ua.get_freq_df(column=train.crew, key='job')
for j in job_df.value[:15]:
    train['crew_job_'+c] = train.crew.apply(lambda x: c in x)
    test['crew_job_'+c] = test.crew.apply(lambda x: c in x)
dept_df = ua.get_freq_df(column=train.crew, key='department')
for d in dept_df.value[:15]:
    train['crew_dept_'+c] = train.crew.apply(lambda x: c in x)
    test['crew_dept_'+c] = test.crew.apply(lambda x: c in x)
    
train['production_companies_size'] = train.production_companies.apply(ua.how_many)
company_freq = ua.get_freq_col(train.production_companies)
production_score = company_freq/train.production_companies_size
train['production_score_log'] = production_score.apply(np.log1p)
company_df = ua.get_freq_df(column=train.production_companies)
test['production_companies_size'] = test.production_companies.apply(ua.how_many)
company_freq = ua.get_freq_col(test.production_companies)
production_score = company_freq/test.production_companies_size
test['production_score_log'] = production_score.apply(np.log1p)
for c in company_df.value[:10].values:
    train['production_company_'+c] = train.production_companies.apply(lambda x: c in x)
    test['production_company_'+c] = test.production_companies.apply(lambda x: c in x)

train['production_countries_size'] = train.production_countries.apply(ua.how_many)
test['production_countries_size'] = test.production_countries.apply(ua.how_many)
country_df = ua.get_freq_df(column=train.production_countries)
for c in country_df.value[:10].values:
    train['production_country_'+c] = train.production_countries.apply(lambda x: c in x)
    test['production_country_'+c] = test.production_countries.apply(lambda x: c in x)

train['genres_size'] = train.genres.apply(ua.how_many)
test['genres_size'] = test.genres.apply(ua.how_many)
genre_df = ua.get_freq_df(column=train.genres)
for genre in genre_df.value:
    train['genre_'+genre] = train.genres.apply(lambda x: genre in x)
    test['genre_'+genre] = train.genres.apply(lambda x: genre in x)
    
train['language_size'] = train.spoken_languages.apply(ua.how_many)
train['english'] = train.original_language.apply(lambda x: x=='en')
test['language_size'] = test.spoken_languages.apply(ua.how_many)
train['english'] = train.original_language.apply(lambda x: x=='en')

train['keyword_size'] = train.Keywords.apply(ua.how_many)
test['keyword_size'] = test.Keywords.apply(ua.how_many)    
keyword_df = ua.get_freq_df(column=train.Keywords)
for k in keyword_df.value[:30].values:
    train['keyword_'+k] = train.Keywords.apply(lambda x: k in x)
    test['keyword_'+k] = test.Keywords.apply(lambda x: k in x)
    
# there are a lot of words, so choosing ones that have the most influence over revenue
overview_important = ba.get_important_words(train.overview, train.revenue_log)
# overview_important.value.describe()
# count    54.000000
# mean     27.703704
# std      10.424263
# min      20.000000
# 25%      22.000000
# 50%      24.000000
# 75%      31.000000
# max      90.000000

for word in overview_important.value:
    train['overview_'+word] = train.overview.apply(lambda x: word in x)
    test['overview_'+word] = test.overview.apply(lambda x: word in x)
    
tagline_important = ba.get_important_words(train.tagline, train.revenue_log)
# tagline_important.value.describe() 
# count    6.000000
# mean     2.333333
# std      0.516398
# min      2.000000
# 25%      2.000000
# 50%      2.000000
# 75%      2.750000
# max      3.000000
# tagline words are too sparse to have much effect

In [12]:
# feature generation - numerical

# fill numeric nans
train.runtime = train.runtime.replace('',0)
train.runtime = train.runtime.replace(0, train.runtime.median())
test.runtime = test.runtime.replace('',0)
test.runtime = test.runtime.replace(0, test.runtime.median())

train.production_score_log.fillna(train.production_score_log.median(), inplace=True)
test.production_score_log.fillna(test.production_score_log.median(), inplace=True)

for g in [0,1,2]:
    train['cast_gender_'+str(g)] = train['cast_gender_'+str(g)].fillna(train['cast_gender_'+str(g)].median())
    test['cast_gender_'+str(g)] = test['cast_gender_'+str(g)].fillna(test['cast_gender_'+str(g)].median())
    train['crew_gender_'+str(g)] = train['crew_gender_'+str(g)].fillna(train['crew_gender_'+str(g)].median())
test['crew_gender_'+str(g)] = test['crew_gender_'+str(g)].fillna(test['crew_gender_'+str(g)].median())

# feature selection

# make sure numeric columns are actually numeric
num_cols = train.select_dtypes(include=[np.float, np.int]).columns.tolist()
# drop features from which log transforms have been generated
num_cols.remove('budget')
num_cols.remove('popularity')
num_cols.remove('revenue_log')
not_to_include = ['id',
 'belongs_to_collection',
 'revenue',
 'release_month',
 'release_weekday']
train[not_to_include] = train[not_to_include].astype(str)

# num_features = select_numeric_features(train, train.revenue_log)
# bool_features = select_bool_features(train, train.revenue_log)

bool_cols = train.select_dtypes(include=[np.bool]).columns.tolist()
object_cols = train.select_dtypes(include=[np.object]).columns.tolist()

train[bool_cols] = train[bool_cols].applymap(lambda x: 1 if x else 0)

In [85]:
train_num = train[num_cols+bool_cols]
target = pd.DataFrame(train.revenue_log)

In [86]:
# scale target
target_scaler = StandardScaler()
target_scaled = pd.DataFrame(target_scaler.fit_transform(target))
# scale numeric data 
num_scaler = StandardScaler()
train_num_scaled = train_num
train_num_scaled[num_cols] = num_scaler.fit_transform(train_num[num_cols])

In [87]:
# clean some more

# drop rows with values > 3 standard deviations from the mean
train_num_scaled['revenue_log'] = target_scaled

train_num_scaled = train_num_scaled[target_scaled.values > -3]

for col in num_cols:
    if (train_num_scaled[col].values < -3).any() or (train_num_scaled[col].values > 3).any():
        train_num_scaled = train_num_scaled[train_num_scaled[col].values > -3]
        train_num_scaled = train_num_scaled[train_num_scaled[col].values < 3]
        
target_scaled = train_num_scaled['revenue_log']
train_num_scaled.drop('revenue_log', axis=1, inplace=True)
target_original = train.revenue[train.revenue.index.isin(target_scaled.index)]

In [88]:
# idk how that got back in there whatever
train_num_scaled.drop('revenue', axis=1, inplace=True)

In [89]:
target_scaled.to_csv('target_scaled.csv')
train_num_scaled.to_csv('train_num_scaled.csv')
target_original.to_csv('target_original.csv')

## preprocessing finished, modeling

In [3]:
train_num_scaled_df = pd.read_csv('train_num_scaled.csv', index_col=0)
target_scaled_df = pd.read_csv('target_scaled.csv', index_col=0, header=None)
target_original = pd.read_csv('target_original.csv', index_col=0, header=None)

In [6]:
target_inverse = target_scaler.inverse_transform(target_scaled_df)
target_inverse = np.expm1(target_inverse)

NameError: name 'target_scaler' is not defined

In [4]:
# choosing features based on f-test. much more reasonable output than using linreg

f, p_val = f_regression(train_num_scaled_df, target_scaled_df)

f_scores = zip(f, p_val, train_num_scaled_df.columns)
f_scores = sorted(f_scores, reverse=True)

f_importances_df = pd.DataFrame({'f': f, 'p_val': p_val, 'feature': train_num_scaled_df.columns})
f_importances_df = f_importances_df.sort_values(by='f', ascending=False)

In [5]:
train_f_significant = train_num_scaled_df[f_importances_df['feature'][f_importances_df['p_val'] < 0.001]]

In [10]:
# linreg = LinearRegression().fit(train_f_significant, target_scaled_df)
# linreg.score(train_f_significant, target_scaled_df)

# pred = linreg.predict(train_f_significant)
# math.sqrt(mean_squared_error(pred, target_scaled_df))

0.6203528858030684

In [6]:
train_f_significant.to_csv('train_f_significant.csv')

shouldn't have to touch anything above this

In [ ]:
train_f_significant = pd.read_csv('train_f_significant.csv', index_col=1)
target_scaled_df = pd.read_csv('target_scaled.csv', index_col=0, header=None)
target_original = pd.read_csv('target_original.csv', index_col=0, header=None)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(train_f_significant, target_scaled_df)
y_tr_original = target_original[target_original.index.isin(y_train.index)]
y_val_original = target_original[target_original.index.isin(y_val.index)]

# linreg.fit(x_train, y_train)
# pred = linreg.predict(x_val)

# pred_inv = target_scaler.inverse_transform(pred)
# pred_inv = np.expm1(pred_inv)

# math.sqrt(mean_squared_log_error(pred_inv, y_val_original))

In [ ]:
gls = GeneticLocalSearch()
best_model = gls.fit(x_train, y_train)

Initializing population
Training population
....................................................................................................
Generation 1:
Selecting parents
Making children
Training population
....................................................................................................
Generation 2:
Selecting parents
Making children
Training population
....................................................................................................
Generation 3:
Selecting parents
Making children
Training population
....................................................................................................
Generation 4:
Selecting parents
Making children
Training population
....................................................................................................
Generation 5:
Selecting parents
Making children
Training population
....................................................................................................
Generation

In [ ]:
print('a')

In [ ]:
# todo: get original target data. get scaler and reverse transform pred (exp(y)-1), calcualte rmsle
# implement iterated local search; determine a good perturbation value
# implement genetic local search
# find good perturbation values
# each new coef is a gaussian random value centered on the old coef 
# with sd some percentage of total distribution sd